In [4]:
import pymongo as pm
import json

## Stimuli Upload
`ratchet_id` refers to a set of rules to be queried for trials. Each collection of `ratchet_id` has a `_dev` version, except for `dev` collection itself.

Current Values:
1. `dev`: 2 single features rules, 2 conjunctive rules, 2 disjunctive rules, 2 conjunctive disjunction rules, & 2 disjunctive conjunction rules
2. `50_rules`: 10 single feature rules, 10 conjunctive rules, 10 disjunctive rules, 10 conjunctive disjunction rules, & 10 disjunctive conjunction rules

In [10]:
def gen_vars(local_machine=True):
    if local_machine:
        pswd = ""
        user = ""
        host = "localhost"
    else:
        auth = pd.read_csv('.auth.txt', header = None)
        pswd = auth.values[0][0]
        user = 'sketchloop'
        host = 'rxdhawkins.me' 
    return user, pswd, host

In [11]:
def convert_concept_summary_into_db_data(data):
    db_data = {
        "SINGLE_FEATURE": [],
        "CONJUNCTION": [],
        "DISJUNCTION": [],
        "CONJUNCTION_DISJUNCTION": [],
        "DISJUNCTION_DISJUNCTION": [],
    }
    for rule_idx, rule in data.items():
        rule_type = rule["type"]
        speciesName = get_creature_names(rule_type)[0]
        speciesNamePlural = get_creature_names(rule_type)[1]
        db_data[rule_type].append({
            "rule_idx": int(rule_idx),
            "file_name": rule["name"],
            "name": rule["phrase"],
            "games": [],
            "numGames": 0,
            "speciesName": speciesName,
            "speciesNamePlural": speciesNamePlural,
        })
    return db_data

In [12]:
def load_data_into_db(concept_summary_file, col_prefix):
    with open(concept_summary_file) as f:
        data = json.load(f)
        db_data = convert_concept_summary_into_db_data(data)
        for rule_type, rules in db_data.items():
            col = col_prefix.format(rule_type)
            for rule in rules:
                db[col].insert_one(rule)

In [15]:
# Set Variables 
user, pswd, host = gen_vars(local_machine=True)

if user == '' or pswd == '':
    conn = pm.MongoClient('mongodb://{}'.format(host))    
else:
    conn = pm.MongoClient('mongodb://{}:{}@{}'.format(user, pswd, host))
db = conn['genGames']


In [16]:
# Load Development Data
dev_col_prefix = 'dev_{}'
dev_concept_summary_file = './dev/concept_summary.json'
load_data_into_db(dev_concept_summary_file, dev_col_prefix)

NameError: global name 'gen_creature_names' is not defined

In [8]:
# Examine which collections currently exist
sorted(db.list_collection_names())

[u'dev_CONJUNCTION',
 u'dev_SINGLE_FEATURE',
 u'pilot_CONJUNCTION',
 u'pilot_SINGLE_FEATURE',
 u'system.indexes']

In [9]:
db['dev_CONJUNCTION'].drop()
db['dev_SINGLE_FEAUTURE'].drop()
db['pilot_CONJUNCTION'].drop()
db['pilot_SINGLE_FEAUTURE'].drop()